In [1]:
import os
import pymssql
import logging
from manager import dms_data_manager
from logging.handlers import TimedRotatingFileHandler
from tqdm import tqdm
import pandas as pd
import math

### Set the logger
logger = logging.getLogger(__name__)
formatter = logging.Formatter("[%(asctime)s.%(msecs)03d] %(levelname)s [%(thread)d] - %(message)s", "%Y-%m-%d %H:%M:%S")

if os.path.isdir('c:\\Logfiles'):
    handler = TimedRotatingFileHandler('c:\\Logfiles\\dms.log', when="midnight", interval=1, encoding="utf-8", backupCount=9)
else:    
    handler = TimedRotatingFileHandler('.\\logs\\dms.log', when="midnight", interval=1, encoding="utf-8", backupCount=9)

logger.setLevel(logging.DEBUG)
logger.addHandler(handler)
handler.setFormatter(formatter)

In [2]:
from manager import dms_data_manager
config = {
    'creator': pymssql,
    'host': '10.71.253.162',
    'port': 1433,
    'user': 'dmsai',
    'password': 'tpcKV2@U',
    'charset': 'utf8',
    'database': 'DMS',
    'maxconnections': 10  # 連接池最大連接數量
}
# Loading data 
use_cols = ['ID','BATCH_NO','ORDER_NO','ORDER_DATE','SHIPPER_CODE','SPECIFY_ARRIVAL_DATE','RECEIVE_ZIP_CODE','CREATE_DT','IS_SA','IS_FREEZE','COD','EST_DISTR_TYPE_ID','REMARK']
DM = dms_data_manager.DMSDataManager(logger, config, use_cols)
df ,DISTR_INFO,ORDER_M_ratio, SHIPPER_INFO, ori_money, holiday, ratio = DM.read_data_from_db(batchNo=None)

{message : return db query result}
{message : return db query result}
{message : return db query result}
{message : return db query result}
{message : return db query result}
{message : return db query result}
{message : return db query result}
{message : return db query result}


In [3]:
from common import rule_based
import json
cfg_path = './config/config.json'
with open(cfg_path, 'r', encoding='utf-8-sig') as f:
    cfg = json.load(f)
# databse connection information
db_cfg = cfg['db_connect']
db_cfg['creator'] = __import__(db_cfg['creator'])
# outlying island information
outly_island_cfg = cfg['islands']
# Run the rule-based program
com_rule = rule_based.rule(df,DISTR_INFO,ORDER_M_ratio, SHIPPER_INFO, ori_money, holiday, ratio, outly_island_cfg,logger)
com_rule.build_limitation()
com_rule.find_cheapest_carrier()
# com_rule.adjustment_ratio()
# DM.generate_result(com_rule.df)

d:\project_in_aroca\common\rule_based.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['EST_DISTR_TYPE_ID'][i] = fin_carrier
d:\project_in_aroca\common\rule_based.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['EST_DISTR_TYPE_ID'][i] = self.df['符合條件配送商'][i][0]
d:\project_in_aroca\common\rule_based.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['EST_DISTR_TYPE_ID'][i] = ''
d:\project_in_aroca\c

In [5]:
print(str(com_rule.df.loc[0,'配送時效']) =='nan')
com_rule.df['REMARK']
# com_rule.df.info()
com_rule.df[com_rule.df['當日隔日'] == '當日隔配']['REMARK']
# com_rule.df

False


0                                                     None
46                                                    None
47                                                    None
291                                                   None
292                                                   None
                               ...                        
18516                                                 None
18523                                                 None
18524                                                 None
18536                                                 None
18545    Unable to find eligible distributors within li...
Name: REMARK, Length: 8218, dtype: object

In [8]:
com_rule.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18552 entries, 0 to 18551
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ORDER_M_ID            18552 non-null  int64         
 1   BATCH_NO              18552 non-null  object        
 2   ORDER_NO              18552 non-null  object        
 3   ORDER_DATE            18552 non-null  object        
 4   SHIPPER_CODE          18552 non-null  object        
 5   SPECIFY_ARRIVAL_DATE  6 non-null      object        
 6   RECEIVE_ZIP_CODE      18552 non-null  object        
 7   CREATE_DT             18552 non-null  datetime64[ns]
 8   IS_SA                 18552 non-null  object        
 9   IS_FREEZE             18552 non-null  object        
 10  COD                   18552 non-null  int64         
 11  EST_DISTR_TYPE_ID     476 non-null    float64       
 12  REMARK                483 non-null    object        
 13  PACK_VOLUME     

In [5]:
com_rule.df['符合條件配送商'].value_counts

<bound method IndexOpsMixin.value_counts of 0                  []
1        [1, 9, 8, 7]
2        [1, 9, 8, 7]
3        [1, 9, 8, 7]
4        [1, 9, 8, 7]
             ...     
18547             NaN
18548             NaN
18549             NaN
18550             NaN
18551             NaN
Name: 符合條件配送商, Length: 18552, dtype: object>

In [12]:
sa_con.reset_index(drop = True)

,DISTR_ID,DISTR_TYPE_ID,DISTR_TYPE_CODE,SHIPPER_TYPE_CODE,COLLECT_WEEK_S,COLLECT_WEEK_E,DISPATCH_DAY,DISPATCH_TIME,ARRIVAL_DAY,ARRIVAL_TIME,DISTR_LIMITATION_CODE,ZIP_CODE,DISTR_COND_CODE,COMPENSATION
0,1,5,b2cSA,affiliateE,1.0,5.0,0,20:00,8,20:00,tmrDelivery,"200,201,202,203,204,205,206,502,500,504,503,50...","cod,sa",150000.0
1,1,5,b2cSA,affiliateE,6.0,6.0,0,12:00,1,20:00,tmrDelivery,"200,201,202,203,204,205,206,502,500,504,503,50...","cod,sa",150000.0


In [10]:
com_rule.DIS_TYPE

{1: [1, 2, 3, 4, 5], 2: [9], 3: [8], 4: [6], 6: [10], 7: [7], 9: [11]}

In [19]:
'affiliateE' in DISTR_INFO[DISTR_INFO['DISTR_COND_CODE'].str.contains('sa','cod')]['SHIPPER_TYPE_CODE'][7]

True

In [6]:
ori_money

,DISTR_TYPE_ID,DISTR_COND_CODE,HOLIDAY_FEE,60,90,110,120,130,150,OTHER,OUTLYINGIS
0,1,normal,None,60.00,66.00,89.00,89.00,107.00,107.00,137.00,250.00
1,2,normal,None,102.00,102.00,102.00,102.00,132.00,132.00,162.00,250.00
2,3,normal,None,72.00,72.00,102.00,102.00,132.00,132.00,162.00,250.00
3,4,normal,None,92.00,92.00,92.00,92.00,122.00,122.00,152.00,250.00
4,5,normal,None,224.00,224.00,224.00,224.00,224.00,224.00,224.00,250.00
5,6,normal,None,120.00,150.00,150.00,190.00,250.00,None,None,275.00
6,7,normal,None,70.00,80.00,100.00,100.00,130.00,130.00,None,220.00
7,8,normal,None,45.00,55.00,55.00,65.00,None,None,None,None
8,9,normal,None,60.00,60.00,80.00,80.00,85.00,85.00,115.00,210.00
9,10,normal,None,65.00,65.00,65.00,115.00,None,None,None,None


In [ ]:
DIS_TYPE = {1:[1,2,3,4],2:[9],3:[8],4:[6],6:[10],7:[7]}
rankk = pd.DataFrame(columns=['DISTR_TYPE_ID','rank'])
for j in DIS_TYPE:
    count = 0
    summ = 0
    for i in range(len(ori_money)):
        if ori_money['DISTR_TYPE_ID'][i] in DIS_TYPE[j]:
            count+=1
            summ+= ori_money['60'][i]
    rankk.loc[len(rankk.index)] = [j,summ/count]
ori_money
rankk
test = pd.merge(com_rule.df1,rankk, on = 'DISTR_TYPE_ID',how="left")

,DISTR_TYPE_ID,rank
0,1,81.50
1,2,60.00
2,3,45.00
3,4,120.00
4,6,65.00
5,7,70.00


In [15]:
test

,DISTR_ID,count,ASSIGN_LIMIT_LOWER,ASSIGN_LIMIT_UPPER,Adjustment,Adjustment_U,rank_x,rank_y
0,7.0,0.0,0.0,24.0,0.0,24.0,0,70.00
1,4.0,0.0,0.0,24.0,0.0,24.0,1,120.00
2,1.0,0.0,0.0,24.0,0.0,24.0,2,81.50
3,2.0,0.0,7.2,24.0,7.2,24.0,3,60.00
4,6.0,0.0,0.0,24.0,0.0,24.0,4,65.00
5,3.0,24.0,4.8,24.0,-19.2,0.0,5,45.00


In [4]:
com_rule.df1

,DISTR_ID,count,ASSIGN_LIMIT_LOWER,ASSIGN_LIMIT_UPPER,Adjustment,Adjustment_U,rank
0,7.0,0.0,0.0,24.0,0.0,24.0,0
1,4.0,0.0,0.0,24.0,0.0,24.0,1
2,1.0,0.0,0.0,24.0,0.0,24.0,2
3,2.0,0.0,7.2,24.0,7.2,24.0,3
4,6.0,0.0,0.0,24.0,0.0,24.0,4
5,3.0,24.0,4.8,24.0,-19.2,0.0,5


In [5]:
com_rule.ratio_dic

{'ASSIGN_LIMIT_LOWER': {1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 6.0: 0, 7.0: 0},
 'ASSIGN_LIMIT_UPPER': {1.0: 24, 2.0: 24, 3.0: 24, 4.0: 24, 6.0: 24, 7.0: 24}}

In [6]:
df['EST_DISTR_TYPE_ID'].value_counts()

8    24
Name: EST_DISTR_TYPE_ID, dtype: int64

In [5]:
com_rule.df

,ORDER_M_ID,BATCH_NO,ORDER_NO,ORDER_DATE,SHIPPER_CODE,SPECIFY_ARRIVAL_DATE,RECEIVE_ZIP_CODE,CREATE_DT,IS_SA,IS_FREEZE,...,貨主歸類,賠償上限,出車星期,預估出車時間,預估出車時間(日),訂單下單星期,訂單下單時段,出車時間,當日隔日,符合條件配送商
0,1,EST_20220825_0826,S2022082510213,2022-08-25,02,None,300,2023-01-09 18:31:35.867,N,N,...,affiliateE,20000.0,1,20:00,0.0,1,18:31,2023/01/09,當日,"[3, 9, 7]"
1,2,202208251335,S2022082510787,2022-08-25,26,None,300,2023-01-09 18:31:36.247,N,N,...,thirdPartyE,5000.0,2,17:00,8.0,1,18:31,2023/01/10,隔日,"[1, 9, 8, 7]"
2,3,202208251335,S2022082510788,2022-08-25,26,None,114,2023-01-09 18:31:36.393,N,N,...,thirdPartyE,5000.0,2,17:00,8.0,1,18:31,2023/01/10,隔日,"[1, 9, 8, 7]"
3,4,202208251335,S2022082510789,2022-08-25,26,None,241,2023-01-09 18:31:36.533,N,N,...,thirdPartyE,5000.0,2,17:00,8.0,1,18:31,2023/01/10,隔日,"[1, 9, 8, 7]"
4,5,202208251335,S2022082510794,2022-08-25,26,None,251,2023-01-09 18:31:36.647,N,N,...,thirdPartyE,5000.0,2,17:00,8.0,1,18:31,2023/01/10,隔日,"[1, 9, 8, 7]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18540,18472,20220925000000827,S2022092510000,2022-09-24,03,None,310,2023-01-09 22:04:22.937,N,N,...,affiliateE,20000.0,2,07:00,8.0,1,22:04,2023/01/10,當日,[6]
18541,18473,20220925001000382,S2022092510010,2022-09-25,03,None,412,2023-01-09 22:04:23.613,N,N,...,affiliateE,20000.0,2,07:00,8.0,1,22:04,2023/01/10,當日,[6]
18542,18474,20220925011000399,S2022092510013,2022-09-25,03,None,242,2023-01-09 22:04:24.320,N,N,...,affiliateE,20000.0,2,07:00,8.0,1,22:04,2023/01/10,當日,"[2, 6, 10]"
18543,18475,20220925012000529,S2022092510014,2022-09-25,03,None,242,2023-01-09 22:04:29.147,N,N,...,affiliateE,20000.0,2,07:00,8.0,1,22:04,2023/01/10,當日,"[2, 6, 10]"


In [95]:
import numpy as np
DIS_TYPE = {1:[1,2,3,4],2:[9],3:[8],4:[6],6:[10],7:[7]}
group = self.ratio.groupby('ID')
# self.df['符合條件配送商'] = self.df['符合條件配送商'].astype('str')
self.df1 = pd.DataFrame(columns=['DISTR_ID','count','ASSIGN_LIMIT_LOWER','ASSIGN_LIMIT_UPPER','Adjustment','Adjustment_U'])
for i in list(group.size().keys()):
    count_dis = 0
    try:
        for j in range(len(DIS_TYPE[i])):
            count_dis += self.df[self.df['EST_DISTR_TYPE_ID']==DIS_TYPE[i][j]]['EST_DISTR_TYPE_ID'].count()
        self.df1.loc[len(self.df1.index)] = [i,count_dis,((int(group.get_group(i)['ASSIGN_LIMIT_LOWER'])/100)*self.df.count()['EST_DISTR_TYPE_ID']),int((group.get_group(i)['ASSIGN_LIMIT_UPPER'])/100*self.df.count()['EST_DISTR_TYPE_ID']),None,None]
    except:
        pass
group3 = self.df1.groupby('DISTR_ID')
for i in list(group3.size().keys()):
    if int(group3.get_group(i)['count']) < int(group3.get_group(i)['ASSIGN_LIMIT_LOWER']):
        self.df1.loc[int(self.df1.loc[self.df1['DISTR_ID']==i].index.values),'Adjustment'] = int(group3.get_group(i)['ASSIGN_LIMIT_LOWER']) - int(group3.get_group(i)['count'])
    elif int(group3.get_group(i)['count']) > int(group3.get_group(i)['ASSIGN_LIMIT_UPPER']):
        self.df1.loc[int(self.df1.loc[self.df1['DISTR_ID']==i].index.values),'Adjustment_U'] = int(group3.get_group(i)['ASSIGN_LIMIT_UPPER']) - int(group3.get_group(i)['count'])
    else:
        self.df1.loc[int(self.df1.loc[self.df1['DISTR_ID']==i].index.values),'Adjustment_U'] = int(group3.get_group(i)['ASSIGN_LIMIT_UPPER']) - int(group3.get_group(i)['count'])
        self.df1.loc[int(self.df1.loc[self.df1['DISTR_ID']==i].index.values),'Adjustment'] = int(group3.get_group(i)['ASSIGN_LIMIT_LOWER']) - int(group3.get_group(i)['count'])
self.df1['rank'] = [2,3,5,1,4,0]
self.df1 = self.df1.sort_values('rank')
self.df1 = self.df1.reset_index(drop = True)
## 要加一個有沒有滿足的dictonary
self.ratio_dic = {'ASSIGN_LIMIT_LOWER':{},'ASSIGN_LIMIT_UPPER':{}}
name = range(len(list(self.df1[self.df1['Adjustment']<=0]['DISTR_ID'].values)))
name_list = list(self.df1[self.df1['Adjustment']<=0]['DISTR_ID'].values)
conn = self.df1[self.df1['Adjustment']<=0].copy()
for i in name:
    cond = abs(int(conn[conn['DISTR_ID']==name_list[i]]['Adjustment']))
    count = 0
    count_1 = 0
    for j in tqdm(range(len(self.df))):

        count_1+=1
        if count > cond:
            self.logger.info('變更配送商：{}'.format(name_list[i]))
            self.logger.info('變更訂單數量：{}'.format(count))
            break
        elif count_1 == len(self.df):
            self.logger.info('變更配送商：{}'.format(name_list[i]))
            self.logger.info('變更訂單數量：{}'.format(count))
        elif self.df.loc[j,'EST_DISTR_TYPE_ID'] in DIS_TYPE[int(name_list[i])]:
            chang_1 = 999
            for k in range(len(DIS_TYPE[self.df1[self.df1['Adjustment']<=0]['DISTR_ID'].values[0]])):
                list_des = self.df.loc[j]['符合條件配送商'].copy()
                list_des = ''.join(str(x) for x in list_des)
                list_chang = list(filter(lambda uu: str(DIS_TYPE[self.df1[self.df1['Adjustment']<=0]['DISTR_ID'].values[0]][k]) not in uu,str(list_des)))
                list_chang = list(set(list_chang)&set(list(filter(lambda uu: str(k) not in uu,list_des))))
            for p in list(self.df1[self.df1['Adjustment']<=0]['DISTR_ID'].values)[1:]:
                # print(DIS_TYPE[int(p)])
                for y in DIS_TYPE[int(p)]:
                    # print(y)
                    list_chang = list(set(list_chang)&set(list(filter(lambda uu: str(y) not in uu,list_des))))         
            for q in list_chang:
                try:
                    volume_compare = int(self.ori_money[self.ori_money['DISTR_TYPE_ID'] == self.df.loc[j]['EST_DISTR_TYPE_ID']][str(self.df.loc[j,'才績級距'])])
                    volume = int(self.ori_money[self.ori_money['DISTR_TYPE_ID'] == q][str(self.df.loc[j]['才績級距'])])
                    chang = volume - volume_compare
                except:
                    continue
                if chang<chang_1:
                    self.df1.loc[(self.df1[self.df1['DISTR_ID'] == name_list[i]].index),'count'] = int(self.df1.loc[(self.df1[self.df1['DISTR_ID'] == list(conn['DISTR_ID'].values)[i]].index),'count'])-1
                    self.df1.loc[(self.df1[self.df1['DISTR_ID'] == name_list[i]].index),'Adjustment'] = int(self.df1.loc[(self.df1[self.df1['DISTR_ID'] == list(conn['DISTR_ID'].values)[i]].index),'Adjustment'])+1
                    self.df1.loc[(self.df1[self.df1['DISTR_ID'] == name_list[i]].index),'Adjustment_U'] = int(self.df1.loc[(self.df1[self.df1['DISTR_ID'] == list(conn['DISTR_ID'].values)[i]].index),'Adjustment_U'])+1
                    chang_1 = chang
                    self.df.loc[j,'EST_DISTR_TYPE_ID'] = q
                    for s in list(group.size().keys()):
                        if s in q:
                            ccc = s
                    self.df1.loc[(self.df1[self.df1['DISTR_ID'] == ccc].index),'count'] = int(self.df1.loc[(self.df1[self.df1['DISTR_ID'] == ccc].index),'count'])+1
                    self.df1.loc[(self.df1[self.df1['DISTR_ID'] == ccc].index),'Adjustment'] = int(self.df1.loc[(self.df1[self.df1['DISTR_ID'] == ccc].index),'Adjustment'])-1
                    self.df1.loc[(self.df1[self.df1['DISTR_ID'] == name_list[i]].index),'Adjustment_U'] = int(self.df1.loc[(self.df1[self.df1['DISTR_ID'] == list(conn['DISTR_ID'].values)[i]].index),'Adjustment_U'])-1
                    count+=1
for i in list(group3.size().keys()):
    if int(group3.get_group(i)['count']) >= int(group3.get_group(i)['ASSIGN_LIMIT_LOWER']):
        self.ratio_dic['ASSIGN_LIMIT_LOWER'][i] = int(group3.get_group(i)['ASSIGN_LIMIT_LOWER'])
    #         # self.df.loc[j,'符合條件配送商'] = ''.join(self.df.loc[j,'符合條件配送商'])if int(group3.get_group(i)['count']) <= int(group3.get_group(i)['ASSIGN_LIMIT_UPPER']):
        self.ratio_dic['ASSIGN_LIMIT_UPPER'][i] = int(group3.get_group(i)['ASSIGN_LIMIT_UPPER'])
self.logger.info('Order Distribution Status Statistics')
for i in range(len(self.df1)):
    self.logger.info('{}:{}'.format(self.df1.loc[i]['DISTR_ID'],self.df1.loc[i]['count']))

100%|██████████| 24/24 [00:00<00:00, 7997.40it/s]


In [5]:
com_rule.df1

,DISTR_ID,count,ASSIGN_LIMIT_LOWER,ASSIGN_LIMIT_UPPER,Adjustment,Adjustment_U,rank
0,7.0,0.0,0.0,24.0,0.0,24.0,0
1,4.0,0.0,0.0,24.0,0.0,24.0,1
2,1.0,0.0,0.0,24.0,0.0,24.0,2
3,2.0,0.0,0.0,24.0,0.0,24.0,3
4,6.0,0.0,0.0,24.0,0.0,24.0,4
5,3.0,24.0,0.0,24.0,-24.0,0.0,5


In [6]:
com_rule.ratio_dic

{'ASSIGN_LIMIT_LOWER': {1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 6.0: 0, 7.0: 0},
 'ASSIGN_LIMIT_UPPER': {1.0: 24, 2.0: 24, 3.0: 24, 4.0: 24, 6.0: 24, 7.0: 24}}

In [11]:
##賠償 郵遞區號 貨主歸類 時效 出車時間 週六日配送要周六日到 
# SA 訂單欄位名稱 離島
list_d = []
if com_rule.df['SA份數'][2135] != 0:
    sa_con = DISTR_INFO[DISTR_INFO['DISTR_COND_CODE'].str.contains('sa')]
    try:
        for j in range(len(sa_con)):
            if str(int(com_rule.df.loc[2135, '郵遞區號'])) in sa_con.loc[j, 'ZIP_CODE'] and com_rule.df.loc[2135, '貨主歸類'] in sa_con.loc[j, 'SHIPPER_TYPE_CODE']:
                print('haha')
                list_d.append(sa_con.loc[j, 'DISTR_TYPE_ID'])
                continue
    except:
        pass
    # continue
elif com_rule.df['代收金額'][2135] != 0:
    cod_con = DISTR_INFO[DISTR_INFO['DISTR_COND_CODE'].str.contains('cod')]
    try:
        for j in range(len(cod_con)):
            if str(int(com_rule.df.loc[2135, '郵遞區號'])) in cod_con.loc[j, 'ZIP_CODE'] and com_rule.df.loc[2135, '貨主歸類'] in cod_con.loc[j, 'SHIPPER_TYPE_CODE']:
                if cod_con.loc[j, 'DISTR_TYPE_ID'] not in list_d:
                    list_d.append(cod_con.loc[j, 'DISTR_TYPE_ID'])
                continue
    except:
        pass
    # continue
con_d = DISTR_INFO.loc[(com_rule.df.loc[2135, '出車星期']>=DISTR_INFO['COLLECT_WEEK_S']) & (com_rule.df.loc[2135, '出車星期']<=DISTR_INFO['COLLECT_WEEK_E']) & (com_rule.df.loc[2135, '預估出車時間']<=DISTR_INFO['DISPATCH_TIME']) & (com_rule.df.loc[2135, '賠償上限']<=DISTR_INFO['COMPENSATION'])]
con_d = con_d.reset_index(drop=True)
list_today = []
list_hol = []
for j in range(len(con_d)):
    if str(int(com_rule.df.loc[2135, '郵遞區號'])) in con_d.loc[j, 'ZIP_CODE'] and com_rule.df.loc[2135, '貨主歸類'] in con_d.loc[j, 'SHIPPER_TYPE_CODE'] and con_d.loc[j, 'DISTR_LIMITATION_CODE'] in com_rule.df.loc[2135, '配送時效']:
        if com_rule.df.loc[2135, '當日隔日'] == '當日' and (con_d.loc[j, 'DISTR_LIMITATION_CODE'][-7:] == 'SameDay' or con_d.loc[j, 'DISTR_LIMITATION_CODE'][-7:] == 'Express'):
            if con_d.loc[j, 'DISTR_TYPE_ID'] not in list_today:
                list_today.append(con_d.loc[j, 'DISTR_TYPE_ID'])
        if com_rule.df.loc[2135, '出車星期'] == 5 and 'satDelivery' in com_rule.df.loc[2135, '配送時效']:
            if con_d.loc[j,'ARRIVAL_DAY']!= 0 and con_d.loc[j,'ARRIVAL_DAY']!= 8:
                if con_d.loc[j, 'DISTR_TYPE_ID'] not in list_d:
                    list_d.append(con_d.loc[j, 'DISTR_TYPE_ID'])
                # continue
            else:
                if con_d.loc[j, 'DISTR_TYPE_ID'] not in list_hol:
                    list_hol.append(con_d.loc[j, 'DISTR_TYPE_ID'])
        elif com_rule.df.loc[2135, '出車星期'] == 6 and 'sunDelivery' in com_rule.df.loc[2135, '配送時效']:
            if con_d.loc[j,'ARRIVAL_DAY']!= 0 and con_d.loc[j,'ARRIVAL_DAY']!= 8:
                if con_d.loc[j, 'DISTR_TYPE_ID'] not in list_d:
                    list_d.append(con_d.loc[j, 'DISTR_TYPE_ID'])
                # continue
            else:
                if con_d.loc[j, 'DISTR_TYPE_ID'] not in list_hol:
                    list_hol.append(con_d.loc[j, 'DISTR_TYPE_ID'])
        else:
            if con_d.loc[j, 'DISTR_TYPE_ID'] not in list_d:
                list_d.append(con_d.loc[j, 'DISTR_TYPE_ID'])
if list_today !=[]:
    com_rule.df.at[2135,'符合條件配送商'] = list_today
elif list_hol !=[]:
    com_rule.df.at[2135,'符合條件配送商'] = list_hol
else:
    com_rule.df.at[2135,'符合條件配送商'] = list_d

        
print(list_d)
print(list_today)
print(list_hol)

[3]
[]
[3]
